In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib tk

In [2]:
names=['traj', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 't']
data = pd.read_csv('/Users/alex/Documents/repos/myptv/with_MyPTV/smoothed_trajectories', 
                   delimiter='\t', names = names, header=None)

In [3]:
data = data[data['traj']!=-1]

In [4]:
ids = list(set(data['traj']))
# ids.remove(-1)
len(ids)

30747

In [5]:
longs = []
for i in ids:
    tr = data[data['traj']==i]
    if len(tr) > 25:
        longs.append(tr)
        
print(len(longs))

3252


In [6]:
from mpl_toolkits import mplot3d
import matplotlib
cmap = matplotlib.cm.get_cmap('winter')
import matplotlib.pyplot as plt


fig = plt.figure()
fig.set_size_inches(9,7)
ax = fig.add_subplot(projection='3d')


for tr in longs:
    V = np.mean(tr['vx']**2 + tr['vy']**2 + tr['vz']**2)**0.5
    color = cmap(V/0.5)
    ax.plot(tr['x'], tr['z'],  tr['y'], color=color)

ax.set_xlim(-40,40)
ax.set_zlim(-40,40)
ax.set_ylim(-40,40)

plt.tight_layout()
# fig.savefig('trajs.pdf')

# Trying a 3D flowing arrows movie:

In [7]:
fig = plt.figure()
fig.set_size_inches(9,8)
ax = plt.axes(projection='3d')
plt.tight_layout()

ax.set_xlim(-40,40)
ax.set_zlim(-40,40)
ax.set_ylim(-40,40)

ax.grid(False)
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

matplotlib.rc('axes',edgecolor='w')

ax.tick_params(axis='x', colors='w')
ax.tick_params(axis='y', colors='w')
ax.tick_params(axis='z', colors='w')

ax.w_xaxis.line.set_color("w")
ax.w_yaxis.line.set_color("w")
ax.w_zaxis.line.set_color("w")

ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])

ax.set_facecolor((0.1, 0.0, 0.0))
fig.set_facecolor((0.1, 0.0, 0.0))

In [8]:
import matplotlib 
cmap = matplotlib.cm.get_cmap('winter')


frames = list(set(data['t']))
N = 5


for e,frm in enumerate(frames[:]):
    ax.clear()
    for tr in longs:
        tr_seg = tr[(tr['t']>frm-N) & (tr['t']<frm+N)]
        V = np.mean(tr['vx']**2+tr['vy']**2+tr['vz']**2)**0.5
        ax.plot(tr_seg['x'], tr_seg['z'], tr_seg['y'], color=cmap((V-0.05)/0.25))
        
        
    ax.set_xlim(-40,40)
    ax.set_zlim(-40,40)
    ax.set_ylim(-40,40)
    #ax.set_xlabel('x [mm]')
    #ax.set_ylabel('z [mm]')
    #ax.set_zlabel('y [mm]')
    
    ax.grid(False)
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    
    ax.w_xaxis.line.set_color("w")
    ax.w_yaxis.line.set_color("w")
    ax.w_zaxis.line.set_color("w")

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    
    fig.savefig('./img/im%03d.jpg'%e)

In [ ]:
import numpy as np
import plotly.graph_objects as go

fig = go.Figure(
data=[go.Scatter3d(x=env.xsuc, y=env.ysuc, z=env.zsuc,
mode=“markers”,marker=dict(color=“darkolivegreen”, size=10)),
])




fig = go.Figure(go.Scatter3d(x=env.xsuc, y=env.ysuc, z=env.zsuc, #this is the trace 0
                        mode='markers',marker=dict(color='darkolivegreen', size=10)))
fig.add_scatter3d(x=env.x_par, y=env.y_par, z=env.z_par,  #this is the trace 1
                  mode='markers',marker=dict(color='gold', size=10,symbol='square'))
N = len(env.xsuc)#it must be equal with the len(env.x_par)
frames = [go.Frame(data= [go.Scatter3d(x=env.xsuc[:k+1],
                                 y=env.ysuc[:k+1],
                                 z=env.zsuc[:k+1]),
                          go.Scatter3d(x=env.x_par[:k+1],
                            y=env.y_par[:k+1],
                            z=env.z_par[:k+1])],
                   name=f'frame{k}',
                   traces=[0,1]) for k in range(N)]  # traces =[0,1] tells plotly.js  that the first element in frame data
                                                     #updates trace 0, while the second the trace 1
fig.update(frames=frames)
fig.update_layout(updatemenus=[dict(type='buttons',
                                    buttons=[dict(label='Play',
                                                  method='animate',
                                                  args=[None, 
                                                        dict(frame=dict(redraw=True,
                                                                        fromcurrent=True, 
                                                                        mode=‘immediate’)) ])])])
fig.update_scenes(xaxis=dict(range=[0.2, 3.8],title="X-axis", autorange=False),
                  yaxis=dict(range=[-0.334,+0.334],title="Y-axis", autorange=False),
                  zaxis=dict(range=[0.7, 1.1],title ="Z-axis", autorange=False))
